### controlnet 模型微调
controlnet模型使得用户可以通过施加额外条件，细粒度地控制扩散模型的生成过程。这一技术最初由 Adding Conditional Control to Text-to-Image Diffusion Models 这篇论文提出，并很快地风靡了扩散模型的开源社区。作者开源了 8 个不同的模型，使得用户可以用 8 种条件去控制 Stable Diffusion 模型（包括版本 1 到 5 ）。这 8 种条件包括姿态估计、深度图、边缘图、素描图。

接下来我们将使用 controlnet 来微调我们的 stable diffusion xl模型.

#### Notebook 步骤
1. 导入 boto3, sagemaker python SDK
2. 构建 controlnet fine-tuning 镜像
3. 实现模型微调
   * 配置超参
   * 创建训练任务
4. 测试

#### 1. 导入 boto3, sagemaker python SDK

In [12]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

images_s3uri = 's3://{0}/controlnet-xl/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
controlnet_s3uri = 's3://{0}/stable-diffusion/controlnet/'.format(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


#### 2. 构建 controlnet xl fine-tuning 镜像

In [7]:
!rm -rf sd_controlnet
!mkdir -p sd_controlnet
!cd sd_controlnet && git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 43056, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 43056 (delta 21), reused 21 (delta 14), pack-reused 43021
Receiving objects: 100% (43056/43056), 27.63 MiB | 36.04 MiB/s, done.
Resolving deltas: 100% (31887/31887), done.


In [8]:
%%writefile Dockerfile_controlnet
## You should change below region code to the region you used, here sample is use us-west-2
#From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04

RUN pip install wandb
RUN pip install bitsandbytes
#RUN export TORCH_CUDA_ARCH_LIST="7.5 8.0 8.6" && export FORCE_CUDA="1" && pip install ninja triton==2.0.0.dev20221120 && git clone https://github.com/xieyongliang/xformers.git /tmp/xformers && cd /tmp/xformers && git submodule update --init --recursive && pip install -r requirements.txt && pip install -e . 


ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

Overwriting Dockerfile_controlnet


* build & push docker镜像

In [3]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sd_controlnet_finetuning"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f ./Dockerfile_controlnet .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

#### 3. 模型微调

   * image_uri: ecr仓库中的 docker 镜像地址
   * instance_type: 用于训练任务的实例大小 , 建议使用 ml.g4dn.xlarge, ml.g5.xlarge
   * class_prompt: 提示词类别
   * instance_prompt: 用于你的图片的关键词
   * model_name: 预训练的模型名称
   

In [15]:
%%writefile ./sd_controlnet/train.sh
bash ./train_controlnet_sdxl-h100.sh
# Run this after 1st raise error
pip uninstall torch torchvision
pip install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu118
bash ./train_controlnet_sdxl-h100.sh

Overwriting ./sd_controlnet/train.sh


In [16]:
%%writefile ./sd_controlnet/train_controlnet_sdxl.sh

export WANDB_API_KEY="298b59ce8a416fd45b5fa9ffc17fe72327854e0c"
export WANDB_WATCH="all"
export WANDB_PROJECT="controlnet" 

mkdir -p /tmp/dog
ls -lt ./
chmod 777 ./s5cmd


cd diffusers && pip install -e .
cd examples/controlnet/ && pip install -r requirements_sdxl.txt

curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
sudo apt-get install git-lfs

# Clone Train Dataset(for production)
#git clone https://huggingface.co/datasets/zobnec/controlnet_fs_dataset_df /tmp/dataset/


export MODEL_NAME="stabilityai/stable-diffusion-2-1-base"
export INSTANCE_DIR="/tmp/dataset/"
export OUTPUT_DIR="/tmp/ouput"
#export OUTPUT_DIR="/opt/ml/model/"
export controlnet_s3uri="s3://sagemaker-us-west-2-687912291502/stable-diffusion/controlnet/"

accelerate launch train_controlnet_sdxl.py \
 --pretrained_model_name_or_path=$MODEL_NAME \
 --output_dir=$OUTPUT_DIR \
 --dataset_name="multimodalart/facesyntheticsspigacaptioned" \
 --conditioning_image_column=spiga_seg \
 --image_column=image \
 --caption_column=image_caption \
 --resolution=512 \
 --learning_rate=1e-5 \
 --validation_image "./face_landmarks1.jpeg" "./face_landmarks2.jpeg" "./face_landmarks3.jpeg" \
 --validation_prompt "High-quality close-up dslr photo of man wearing a hat with trees in the background" "Girl smiling, professional dslr photograph, dark background, studio lights, high quality" "Portrait of a clown face, oil on canvas, bittersweet expression" \
 --train_batch_size=4 \
 --num_train_epochs=3 \
 --tracker_project_name="controlnet" \
 --enable_xformers_memory_efficient_attention \
 --checkpointing_steps=500 \
 --validation_steps=500 \
 --report_to="wandb" \
 --push_to_hub


/opt/ml/code/s5cmd sync /tmp/ouput/ $controlnet_s3uri/output/$(date +%Y-%m-%d-%H-%M-%S)/


Writing ./sd_controlnet/train_controlnet_sdxl.sh


   * 创建训练任务

In [ ]:
import time
from sagemaker.estimator import Estimator
from sagemaker.pytorch.estimator import PyTorch

environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'MODEL_NAME':'stabilityai/stable-diffusion-2-1-base'
}

## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region_name, repo_name)
base_job_name = 'sd-xl-controlnet-finetuning-high'
instance_type = 'ml.p4d.24xlarge'
#inputs = {
#    'images': f"s3://{bucket}/controlnet-xl/images/"
#}

estimator = PyTorch(role=role,
                      entry_point='train_controlnet_sdxl.sh',
                      source_dir='./sd_controlnet/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      keep_alive_period_in_seconds=3600, #warmpool，为下一次训练保持机器&镜像（滚动续期，最大1hour）；需要开quota。
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()

In [ ]:
print("Model artifact saved at:\n", controlnet_s3uri)